<a href="https://colab.research.google.com/github/pmy0792/pytorch_tutorial/blob/main/lab_07_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Maximum Likelihood Estimation(MLE) : observation을 가장 잘 설명하는 θ를 찾아내는 과정

Overfitting 방지법
- More data
- Less features
- Regularization

**Regularization**
- Early Stopping: validation loss가 더이상 낮아지지 않을 때
- Reducing Network Size
- Weight Decay: NN의 파라미터 크기 제한
- Dropout
- Batch Normalization

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [36]:
torch.manual_seed(1)

In [37]:
x_train=torch.FloatTensor([[1,2,1], # |x_train| = (m,3)
                           [1,3,2],
                           [1,3,4],
                           [1,5,5],
                           [1,7,5],
                           [1,2,5],
                           [1,6,6],
                           [1,7,7]])

y_train=torch.LongTensor([2,2,2,1,1,1,0,0]) # |y_train| = (m,)

In [38]:
x_test=torch.FloatTensor([[2,1,1], # |x_test| = (m', 3)
                          [3,1,2],
                          [3,3,4]])
y_test=torch.LongTensor([2,2,2]) # |y_test| =(m',)

In [39]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear= nn.Linear(3,3) 
    # nn.Linear(input_dim, output_dim): 선형회귀모델

  def forward(self,x):
    return self.linear(x) # |x| = (m, 3) => (m, 3)

In [40]:
model=SoftmaxClassifierModel()

In [41]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=0.2)

In [42]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):

    # H(x) 계산
    # # model(x_train)은 model.forward(x_train)와 동일
    # orward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행됨 
    # 예를 들어 model이란 이름의 객체를 생성 후, model(입력 데이터)와 같은 형식으로 객체를 호출하면 자동으로 forward 연산이 수행됨
    prediction = model(x_train) # |x_train| = (m,3), |prediction| = (m,3)

    # cost 계산
    cost = F.cross_entropy(prediction,y_train) # |y_train| = (m,)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost:{:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [43]:
def test(model, optimizer, x_test, y_test):
  prediction = model(x_test)
  predicted_classes = prediction.max(1)[1]
  correct_count = (predicted_classes == y_test).sum().item()
  cost = F.cross_entropy(prediction,y_test)

  print('Accuracy: {}% Cost: {:.6f}'.format(
      correct_count / len(y_test) * 100, cost.item()
  ))

In [44]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost:2.203667
Epoch    1/20 Cost:1.840367
Epoch    2/20 Cost:1.584391
Epoch    3/20 Cost:2.069124
Epoch    4/20 Cost:1.743202
Epoch    5/20 Cost:1.320897
Epoch    6/20 Cost:1.694999
Epoch    7/20 Cost:1.769424
Epoch    8/20 Cost:1.394587
Epoch    9/20 Cost:1.566779
Epoch   10/20 Cost:1.785980
Epoch   11/20 Cost:1.501441
Epoch   12/20 Cost:1.469764
Epoch   13/20 Cost:1.710151
Epoch   14/20 Cost:1.480424
Epoch   15/20 Cost:1.430445
Epoch   16/20 Cost:1.754025
Epoch   17/20 Cost:1.509513
Epoch   18/20 Cost:1.344222
Epoch   19/20 Cost:1.653306


In [45]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0% Cost: 0.513143


## Data Preprocessing

In [46]:
x_train=torch.FloatTensor([[73,80,75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

In [47]:
 # data standardization
 mu = x_train.mean(dim=0)
 sigma = x_train.std(dim=0)
 norm_x_train = (x_train - mu) / sigma

In [48]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


## Training with Preprocessed Data

In [55]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(3,1)
  def forward(self,x):
    return self.linear(x)

In [60]:
model = MultivariateLinearRegressionModel()

In [61]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [63]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):

    # H(x) 계산
    prediction = model(x_train) # |x_train| = (m,3), |prediction| = (m,1)

    # cost 계산
    cost = F.mse_loss(prediction,y_train) # |y_train| = (m,)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost:{:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [64]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost:29770.843750
Epoch    1/20 Cost:18898.781250
Epoch    2/20 Cost:12050.423828
Epoch    3/20 Cost:7699.415527
Epoch    4/20 Cost:4924.062012
Epoch    5/20 Cost:3150.534668
Epoch    6/20 Cost:2016.257812
Epoch    7/20 Cost:1290.541992
Epoch    8/20 Cost:826.142700
Epoch    9/20 Cost:528.939758
Epoch   10/20 Cost:338.729218
Epoch   11/20 Cost:216.989227
Epoch   12/20 Cost:139.070099
Epoch   13/20 Cost:89.196045
Epoch   14/20 Cost:57.270782
Epoch   15/20 Cost:36.833179
Epoch   16/20 Cost:23.747566
Epoch   17/20 Cost:15.367709
Epoch   18/20 Cost:9.999623
Epoch   19/20 Cost:6.559284
